In [1]:
import os


In [2]:
%pwd


'f:\\deep-learning-project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\deep-learning-project'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    


In [12]:
from src.cnnclassifier.constants import *
from src.cnnclassifier.utils.common import read_yaml,create_directories


In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [14]:
import os
import urllib.request as request
import zipfile
from cnnclassifier import logger
from cnnclassifier.utils.common import get_size

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-17 19:09:04,800] [INFO] [common.py:32] yaml file: config\config.yaml loaded successfully
[2025-02-17 19:09:04,802] [INFO] [common.py:32] yaml file: params.yaml loaded successfully
[2025-02-17 19:09:04,805] [INFO] [common.py:52] created directory at: artifacts
[2025-02-17 19:09:04,806] [INFO] [common.py:52] created directory at: artifacts/data_ingestion
[2025-02-17 19:09:18,884] [INFO] [4037744573.py:13] artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 43153467
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "b9d9e6a8ee5ca274d7269851c98576fa8abd836edd01bb0e076a209826bda6a7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4304:2D3AE3:2949BB:3E2575:67B33BF7
Accept-Ranges: bytes
Date: Mon, 17 Feb 2025 13:39:09 GMT
Via: 1.1 varnish
X-Ser